In [1]:
library(readxl)
library(timedeppar)

Loading required package: mvtnorm



# Functions

In [19]:
initialize_dataframes <- function() {
    harmonics_cols <- c("Period", "dPeriod", "Amplitude", "dAmplitude", 
              "Phase", "dPhase")
    obs_cols <- c("Age", "d18o")
    
    # load dataframes
    if (is.na(table_stal_harmonics)) {
        table_stal_harmonics <- "stalagmites_30ky_harmonics_original.xlsx"
    }
    if (is.na(table_ice_harmonics)) {
        table_ice_harmonics <- "icecores_30ky_harmonics_original.xlsx"
    }
    if (is.na(table_ice_raw)) {
        table_ice_raw <- "icecores_30ky_raw_original.xlsx"
    }
    
    stal_harmonics <- read_excel(paste0("data/", table_stal_harmonics), skip = 1,
                                 col_names = harmonics_cols, col_types = "numeric")

    d18o_harmonics <- read_excel(paste0("data/", table_ice_harmonics), skip = 1,
                                 col_names = harmonics_cols, col_types = "numeric")

    d18o_raw_tot <- read_excel(paste0("data/", table_ice_raw), skip = 1,
                                 col_names = obs_cols, col_types = "numeric")   
    
    # center raw d18o data and keep just first N years
    years_from_now <- d18o_raw_tot$Age - d18o_raw_tot$Age[1]
    d18o_raw_tot$Age <- years_from_now
    first30ky <- d18o_raw_tot[years_from_now <= n_years,]
    first30ky["d18o"] <- first30ky$d18o - mean(first30ky$d18o)
    d18o_raw <- first30ky[first30ky$Age <= n_years,]
    
    # add new columns
    d18o_harmonics$Omega <- numeric(nrow(d18o_harmonics))
    d18o_harmonics$dOmega <- numeric(nrow(d18o_harmonics))
    d18o_harmonics$FixedOmega <- rep(FALSE, nrow(d18o_harmonics))
    
    # identify reference omegas
    cat("Fixed harmonics:\n")
    for (i in 1:nrow(d18o_harmonics)) {
        for (j in 1:nrow(stal_harmonics)) {
            d18o_period <- d18o_harmonics$Period[i]
            stal_period <- stal_harmonics$Period[j]
            
            # calculate relative difference
            rel_diff <- abs(d18o_period - stal_period) / stal_period
            if (rel_diff <= 3/100) {
                d18o_harmonics$FixedOmega[i] <- TRUE
                d18o_harmonics$Period[i] <- stal_harmonics$Period[j]
                d18o_harmonics$dPeriod[i] <- stal_harmonics$dPeriod[j]
                cat("\nStal. period: ", stal_period, ", d18O period: ", d18o_period, sep="")
            }
        }
    }
    
    # calculate values of Omega and dOmega
    d18o_harmonics$Omega <- 2 * pi / d18o_harmonics$Period
    d18o_harmonics$dOmega <- 2 * pi * d18o_harmonics$dPeriod / d18o_harmonics$Period^2
    
    # return all the dataframes
    list(
        stal_harmonics = stal_harmonics,
        d18o_harmonics = d18o_harmonics,
        d18o_raw = d18o_raw
    )
    
}

initialize_params <- function(dfs, sigma_d18o=1, gamma=15) {
    # TODO: change sigma_d18o
    
    harmonics <- dfs$d18o_harmonics
    params_names <- c()
    
    raw <- dfs$d18o_raw
    time_diffs <- diff(raw$Age)
    
    # define parameters properties
    param.ini <- list()
    param.ou.ini <- c()
    param.range <- list()
    param.log <- c()
    
    # add harmonics' free parameters to "param.ini"
    for (i in 1:nrow(harmonics)) {
        names <- paste0(c("Amplitude", "Phase", "Period"), i)
        
        A <- harmonics$Amplitude[i]
        param.ini[[names[1]]] <- A
        param.ini[[names[2]]] <- harmonics$Phase[i]
        if (!harmonics$FixedOmega[i]) {
            param.ini[[names[3]]] <- harmonics$Period[i]
        }
        
        param.range[[names[1]]] <- c(A - harmonics$dAmplitude[i], A + harmonics$dAmplitude[i])
        param.range[[names[2]]] <- c(0, 2*pi)
        if (!harmonics$FixedOmega[i]) {
            param.range[[names[3]]] <- harmonics$Period[i] * c(0.97, 1.03)
        } else {
            names <- paste0(c("Amplitude", "Phase"), i)
        }
        
        params_names <- c(params_names, names)
        
    }
    param.ini$xi <- cbind(
        t = raw$Age[-nrow(raw)],
        y = time_diffs
    )
    param.ini$sigma_d18o <- sigma_d18o
    
    # add xi parameters to "param.ou.ini"
    param.ou.ini <- c( mean(time_diffs), sd(time_diffs), gamma)
    names(param.ou.ini) <- c("xi_mean", "xi_sd", "xi_gamma")
    
    # ranges
    param.range$sigma_d18o = c(0.01, 10)
    param.range$xi_sd = c(0.01, 10) # TODO: change values
    param.range$xi_gamma = c(1, 20)
    
    # for all free parameters set whether to infer in log-space or not
    param.log <- c(rep(FALSE, length(params_names)+3), TRUE)
    names(param.log) <- c(params_names, "xi", "xi_sd", "xi_gamma", "sigma_d18o")
    
    list(
        ini = param.ini,
        ou.ini = param.ou.ini,
        range = param.range,
        log = param.log
    )
}

log_likelihood <- function(params, data) {
    # build the model from current parameters
    model <- build_model(data, params)$y
    
    # feedback
    count <<- count + 1
    current_percent <- as.integer(100 * count / max_count)
    
    if (current_percent > percent) {        
        # draw percent bar
        percent <<- current_percent
        filled <- floor(percent / 2)
        spaces <- max(0, 50 - filled - 1)
        
        bar <- c(rep("█", filled), rep("░", spaces))
        cat(paste0("\r", paste(bar, collapse = ""), 
                   "  ", current_percent, "%"))
        
        flush.console()
    }
    
    # return the log-likelihood of the observed data
    sum(dnorm(data$d18o, mean = model, 
              sd = params$sigma_d18o, log=TRUE))
}

logprior <- function(params) {    
    harmonics <- dataframes$d18o_harmonics
    ix <- 1:nrow(harmonics)
    
    res <- 0
    for (param_name in c("Amplitude", "Period", "Phase")) {
        if (param_name == "Period") {
            ix <- ix[!(harmonics$FixedOmega)]
        }
        
        param0 <- harmonics[[param_name]][ix]
        param <- params[paste0(param_name, ix)]
        if (param_name == "Period") {
            param <- params[paste0(param_name, ix)]
        }
        dparam <- harmonics[[paste0("d", param_name)]][ix]
        if (param_name == "Phase") {
            dparam <- rep(0.1, length(ix))
        }
        
        res <- res + sum(dnorm(param, mean = param0, 
                               sd = dparam, log = TRUE))
    }
    return(res)
}

run_inference <- function(dfs, params) {    
    percent <<- 0
    count <<- 0
    max_count <<- n_iter * (n_interval + 1)
    
    raw <- dfs$d18o_raw
    
    res <- infer.timedeppar(
        loglikeli = log_likelihood,
        data = raw,
        param.ini = params$ini,
        param.ou.ini = params$ou.ini,
        param.log = params$log,
        param.range = params$range,
        param.logprior = logprior,
        verbose = 0,
        n.iter = n_iter,
        control = list(
            n.interval = n_interval,
            n.adapt = floor(0.4 * n_iter)
        )
    )
    
    # plot(res, type = c("traces", "time-series"))
    return(res)
}

build_model <- function(raw, params=NA) {
    harmonics <- dataframes$d18o_harmonics
    
    times <- raw$Age
    if (!sum(is.na(params))) {
        xi <- params$xi[,2]
        times <- c(raw$Age[1], raw$Age[1] + cumsum(xi))
    }
    
    model <- numeric(length(times))
    for (i in 1:nrow(harmonics)) {
        amplitude <- harmonics$Amplitude[i]
        phase <- harmonics$Phase[i]
        omega <- 2 * pi / harmonics$Period[i]
        
        if (!sum(is.na(params))) {
            amplitude <- params[[paste0("Amplitude", i)]]
            phase <- params[[paste0("Phase", i)]]
            if (!harmonics$FixedOmega[i]) {
                omega <- 2 * pi / params[[paste0("Period", i)]]
            }
        }

        model <- model + amplitude * cos(omega * times + phase)
    }
    
    return(list(x=times, y=model))
}

In [47]:
old <- 'compare_results <- function(res) {
    # init
    df_before <- build_model(dataframes$d18o_raw)
    df_after <- build_model(dataframes$d18o_raw, res$param.maxpost)

    table_name <- ""
    if (is.na(table_stal_harmonics)) {
        table_name <- "stal_original"
    } else {
        table_name <- unlist(strsplit(table_stal_harmonics, "\\."))[1]
    }
    
    if (is.na(table_ice_harmonics)) {
        table_name <- paste0(table_name, "+icecores_original")
    } else {
        table_name <- paste0(table_name, "+", unlist(strsplit(table_ice_harmonics, "\\."))[1])
    }

    path <- file.path("images", paste0(n_iter, " iter"), paste0(n_years, " years"), table_name)
    dir.create(path, recursive = TRUE, showWarnings = FALSE)

    # plots
    plot_comparison(path, dataframes$d18o_raw, df_before, df_after)
    plot_time_difference(path, res$sample.param.timedep$xi)
    plot_time_series(path, res)
    plot_traces(path, res)
}

plot_comparison <- function(path, raw_data, before, after, data_lw=0.5, fit_lw=2.5, col_before="firebrick1",
                            col_after="steelblue", xlim=NULL) {
    if (save_plots) {
        filename <- file.path(path, "d18o_vs_t.jpeg")
        png(filename, width = 1200, height = 1000)
    } else {
        options(repr.plot.width = 12, repr.plot.height = 10)
    }

    layout(matrix(c(1, 1, 2, 3), nrow = 2, byrow = TRUE))
    
    # 1
    plot(raw_data,
         main = "Bayesian Harmonic Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(before,
          col = col_before, lwd = fit_lw
    )
    lines(after,
          col = col_after, lwd = fit_lw
    )
    legend("topleft",
           legend=c("Original data", "Bayesian fit before", "Bayesian fit after"),
           lty=c(1, 1, 1),
           lwd=c(data_lw, fit_lw, fit_lw),
           col=c(1, col_before, col_after),
           #inset = c(-0.05, 0)
    )
    grid()

    # 2
    plot(raw_data,
         main = "Original Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(before,
          col = col_before, lwd = fit_lw
    )
    grid()

    # 3
    plot(raw_data,
         main = "Inferred Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(after,
          col = col_after, lwd = fit_lw
    )
    grid()

    if (save_plots) dev.off()
}

plot_time_difference <- function(path, xi, fit_lw=2.5) {
    if (save_plots) {
        filename <- file.path(path, "time_diff.jpeg")
        png(filename, width = 1200, height = 600)
    }
    
    xi_final <- xi[nrow(xi),]
    t0 <- xi[1,]
    t1 <- c(0, cumsum(xi_final[1:(length(xi_final)-1)]))
    
    plot(t1, t1 - t0, 
         type = "l", lwd = fit_lw, col = "mediumseagreen",
         xlab = "Age", ylab = "Difference in time values",
         main = "Time difference (Delta t vs t)"
    )
    grid()
    
    if (save_plots) dev.off()
}

plot_time_series <- function(path, res) {
    if (save_plots) {
        filename <- file.path(path, "time_series.jpeg")
        png(filename, width = 1200, height = 600)
    }
    
    plot(res, type = "time-series")
    
    if (save_plots) dev.off()
}

plot_traces <- function(path, res) {   
    if (save_plots) {
        filename <- file.path(path, paste0("traces.pdf"))
        pdf(filename, width = 10, height = 6)
    } 
    plot(res, type = "traces")
    if (save_plots) dev.off()
}'

In [42]:
compare_results <- function(res) {
    # init
    df_before <- build_model(dataframes$d18o_raw)
    df_after <- build_model(dataframes$d18o_raw, res$param.maxpost)

    table_name <- ""
    if (is.na(table_stal_harmonics)) {
        table_name <- "stal_original"
    } else {
        table_name <- unlist(strsplit(table_stal_harmonics, "\\."))[1]
    }
    
    if (is.na(table_ice_harmonics)) {
        table_name <- paste0(table_name, "+icecores_original")
    } else {
        table_name <- paste0(table_name, "+", unlist(strsplit(table_ice_harmonics, "\\."))[1])
    }

    path <- file.path("images", paste0(n_iter, " iter"), paste0(n_years, " years"), table_name)
    dir.create(path, recursive = TRUE, showWarnings = FALSE)

    if (save_plots) {
        filename <- file.path(path, paste0("all_plots.pdf"))
        pdf(filename, width = 10, height = 6)
    }

    # plots
    plot_comparison(dataframes$d18o_raw, df_before, df_after)
    plot_time_difference(res$sample.param.timedep$xi)
    plot_time_series(res)

    if (save_plots) dev.off()
}

plot_comparison <- function(raw_data, before, after, data_lw=0.5, fit_lw=2.5, col_before="firebrick1",
                            col_after="steelblue", xlim=NULL) {
    
    # 1
    plot(raw_data,
         main = "Bayesian Harmonic Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(before,
          col = col_before, lwd = fit_lw
    )
    lines(after,
          col = col_after, lwd = fit_lw
    )
    legend("topleft",
           legend=c("Original data", "Bayesian fit before", "Bayesian fit after"),
           lty=c(1, 1, 1),
           lwd=c(data_lw, fit_lw, fit_lw),
           col=c(1, col_before, col_after),
           #inset = c(-0.05, 0)
    )
    grid()

    # 2
    plot(raw_data,
         main = "Original Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(before,
          col = col_before, lwd = fit_lw
    )
    grid()

    # 3
    plot(raw_data,
         main = "Inferred Fit",
         type = "l", lwd = data_lw,
         xlim = xlim
    )
    lines(after,
          col = col_after, lwd = fit_lw
    )
    grid()
}

plot_time_difference <- function(xi, fit_lw=2.5) {
    xi_final <- xi[nrow(xi),]
    t0 <- xi[1,]
    t1 <- c(0, cumsum(xi_final[1:(length(xi_final)-1)]))
    
    plot(t1, t1 - t0, 
         type = "l", lwd = fit_lw, col = "mediumseagreen",
         xlab = "Age", ylab = "Difference in time values",
         main = "Time difference (Delta t vs t)"
    )
    grid()
}

plot_time_series <- function(res) {    
    plot(res, type = c("traces", "time-series"))
}

In [11]:
n_years <- 10000
n_iter <- 10000
n_interval <- 25
#table_stal_harmonics <- "new_results_may31.xlsx"
table_stal_harmonics <- "stalagmites_10ky_harmonics_0601Fourier.xlsx"
table_ice_raw <- NA
#table_ice_harmonics <- "10k_year.xlsx"
table_ice_harmonics <- "icecores_10ky_harmonics_0601Fourier.xlsx"
save_plots <- TRUE

In [4]:
dataframes <- initialize_dataframes()
params <- initialize_params(dataframes)
res <- run_inference(dataframes, params)

Warning message:
“Coercing text to numeric in A2 / R2C1: '10718.378819'”
Warning message:
“Coercing text to numeric in B2 / R2C2: '236.838769'”
Warning message:
“Coercing text to numeric in C2 / R2C3: '0.506526'”
Warning message:
“Coercing text to numeric in D2 / R2C4: '0.033480'”
Warning message:
“Coercing text to numeric in E2 / R2C5: '0.839430'”
Warning message:
“Coercing text to numeric in F2 / R2C6: '0.093580'”
Warning message:
“Coercing text to numeric in A3 / R3C1: '5272.170473'”
Warning message:
“Coercing text to numeric in B3 / R3C2: '130.719141'”
Warning message:
“Coercing text to numeric in C3 / R3C3: '0.150130'”
Warning message:
“Coercing text to numeric in D3 / R3C4: '0.024206'”
Warning message:
“Coercing text to numeric in E3 / R3C5: '2.243977'”
Warning message:
“Coercing text to numeric in F3 / R3C6: '0.229163'”
Warning message:
“Coercing text to numeric in A4 / R4C1: '3424.990249'”
Warning message:
“Coercing text to numeric in B4 / R4C2: '72.217688'”
Warning message:
“C

Fixed harmonics:

Stal. period: 10718.38, d18O period: 10554.89

infer.timedeppar (timedeppar 1.0.3 2023-08-28): starting new Markov Chain

  number of constant parameters:          18

  number of time-dependent parameters:    1

  number of estimated process parameters: 3



█████████████████████████████████████████████████  98%

10000 iterations completed

  acceptance frequency of constant parameters:             0.182

  acceptance frequencies of time-dependent parameters:     0.607

  acceptance frequencies of Ornstein-Uhlenbeck parameters: 0.205



In [43]:
compare_results(res)

plot.timedeppar (timedeppar 1.0.3 2023-08-28): plotting inference results: traces,time-series



pdf 
  2

In [6]:
# TODO: Save plots in res

In [7]:
#plot(res, type = c("traces", "time-series"))